In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from helpers import df_fibonacci_up_to

In [ ]:
fib_data = df_fibonacci_up_to(8, shuffle=True)
fib_data

## Let's train it & vizualize  :)

In [ ]:
## Network configuration
learning_rates = [0.001, 0.0001, 0.00001]

with tf.name_scope("input_data"):
    input_numbers = tf.placeholder(tf.float32, shape=[None, 2], name="input")
    output_numbers = tf.placeholder(tf.float32, shape=[None, 1], name="output")
    learning_rate = tf.placeholder(tf.float32)

with tf.name_scope("layer"):
    weight = tf.Variable(tf.truncated_normal([2, 1]), dtype=tf.float32, name="weight")
    bias = tf.Variable(tf.zeros([1]), dtype=tf.float32, name="bias")
    prediction = tf.matmul(input_numbers, weight) + bias 
    tf.summary.histogram('weight', weight)
    tf.summary.histogram('bias', bias)
    tf.summary.histogram('prediction', prediction)
    
    
with tf.name_scope("evaluation"):
    loss = tf.reduce_mean(tf.square(prediction - output_numbers), name="loss")
    tf.summary.scalar('loss', loss)
    

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(loss)

In [ ]:
epochs = 10000  
feed_dict = {input_numbers: fib_data[['x1', 'x2']], output_numbers: fib_data[['y']]}
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    for lr in learning_rates:
        print('learning rate', lr)
        feed_dict[learning_rate] = lr
        sess.run(init)    
        merged_summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter("logs/02/"+str(lr))
        writer.add_graph(sess.graph)
    
        for epoch in range(epochs):
            if epoch % 400 == 0:
                print('loss', sess.run(loss, feed_dict))
            if epoch % 5 ==0:
                s = sess.run(merged_summary, feed_dict)
                writer.add_summary(s, epoch)
            sess.run(train, feed_dict)
        print(sess.run([weight, bias], feed_dict))
        a = sess.run(prediction, feed_dict)